In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "data/Town_Hall_Rich_List_2023.xlsx"
data_df = pd.read_excel(file).fillna(0)

In [3]:
data_df = data_df \
    .groupby(['Council'], sort=False, as_index=False) \
    .agg(
        count_people=("Council", "count"),
        sum_tc=("Total", "sum"),
        mean_tc=("Total", "mean"),
        sd_tc=("Total", np.std)
).fillna(0)

data_df.to_pickle("pickles/th_2023.pkl")